In [1]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch torchvision torchaudio
!pip install pandas numpy pillow pathlib datasets scikit-learn optuna plotly nbformat
!pip install transformers[torch]
!pip install ipywidgets

  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 59.7/59.7 kB ? eta 0:00:00
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 1.0/199.4 MB 32.7 MB/s eta 0:00:07
   ---------------------------------------- 2.3/199.4 MB 29.1 MB/s eta 0:00:07
    --------------------------------------- 3.2/199.4 MB 25.6 MB/s eta 0:00:08
    --------------------------------------- 4.0/199.4 MB 19.6 MB/s eta 0:00:10
    --------------------------------------- 5.0/199.4 MB 21.2 MB/s eta 0:00:10
   - -------------------------------------- 6.2/199.4 MB 22.0 MB/s eta 0:00:09
   - -------------------------------------- 7.6/199.4 MB 24.5 MB/s eta 0:00:08
   - -------------------------------------- 8.5/199.4 MB 24.8 MB/s eta 0:00:08
   - -------------------------------------- 9.9/199.4 MB 24.4 MB/s eta 0:00:08
   -- ----

In [2]:
import datetime
import json
import numpy as np
import optuna
import pandas as pd
import pickle
import torch

from dataclasses import dataclass
from datasets import load_metric, list_metrics, Dataset
from enum import auto, Enum
from functools import partial
from math import ceil
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset as Dataset_torch
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from torchvision.transforms.functional import (resize, pad, to_tensor, to_pil_image)
from torchvision.transforms import (
    Compose,
    ColorJitter,
    GaussianBlur,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomAffine)
from typing import List


import logging
import sys

In [3]:
class NFCS(Enum):
    SN = auto()
    FS = auto()
    FP = auto()
    BA = auto()
    BE = auto()

@dataclass
class DatasetMeta:
    name: str
    csv_path: str

full_dataset = DatasetMeta(
    name='Full', csv_path=r"D:\ComputerScience\Mestrado\data\UNIFESP\NEW_GT.csv")
unifesp_dataset = DatasetMeta(
    name='UNIFESP', csv_path=r"D:\ComputerScience\Mestrado\data\UNIFESP\NEW_GT_UNIFESP.csv"
)

## Constants

In [4]:
# Datetime
now = datetime.datetime.now()
now_str = now.strftime('%d%m%y-%H%M')
# General
MOCK = False # Marks this as a mock run. Mock runs are saved in a test directory to avoid poluting the results directory
DESCRIPTION = '' # Use this string to describe any important points of the experiment. This will be saved along with the results
# Model
DATASET:DatasetMeta   = full_dataset
NFCS_REGION:NFCS      = NFCS.SN
MASKED:bool           = False # If true use the masked crops if false use the cropped only
VALIDATION_SIZE:float = 0.15 # % of images used for validation
METRICS:List[str]     = ["accuracy", "f1"] # For a list of available metrics call the function list_metrics()
OBJECTIVE_METRIC      = "eval_f1" # Metric that will be optimized by optuna
GREATER_IS_BETTER     = True # Relative to the objective metric
DIRECTION             = "maximize" if GREATER_IS_BETTER else "minimize"
# Hyperparam tunning
NUM_TRIALS:int   = 30 # 20
TRAIN_EPOCHS:int = 20 # 10
# Model path to download from huggingface
model_name_or_path:str = 'google/vit-base-patch16-224-in21k'



# Dataset paths
output_dir = r'D:\ComputerScience\Mestrado\results\Optuna' + f"{'/mock' if MOCK else ''}/ViT/{DATASET.name}/{'non_' if MASKED==False else ''}masked/{NFCS_REGION.name}/{now_str}"
csv_path = DATASET.csv_path
masked_images_path = r"D:\ComputerScience\Mestrado\data\masked_crops"
non_masked_images_path = r"D:\ComputerScience\Mestrado\data\non_masked_crops"
# Create output dirs if needed
Path(output_dir).mkdir(parents=True, exist_ok=True)

---

## Hyperparameter Optimization using Optuna

In [5]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32]),
        "weight_decay": trial.suggest_categorical("weight_decay", [0, 0.1])
    }

def compute_objective(metrics):
    return metrics[OBJECTIVE_METRIC]

---

## Model / Model Pre-Processor

In [6]:
# Processor (pre-process the images to fit the expected input)
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

def model_init(trial):
    # Model with pre-loaded weights and + an untrained classifier head
    model = ViTForImageClassification.from_pretrained(
        model_name_or_path,
        num_labels=2,
        # id2label={str(i): c for i, c in enumerate(labels)},
        # label2id={c: str(i) for i, c in enumerate(labels)}
    )
    # Freeze parameters for the base model (don't freeze the classification head)
    # for param in model.base_model.parameters():
    #     param.requires_grad = False
    
    return model


---

## Augmentations

In [7]:
# Image augmentations
augmentations = Compose([ # https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html
    RandomAffine(degrees=(-45,45), translate=(0.1, 0.3), scale=(0.7, 0.9)),
    ColorJitter(brightness=.4, hue=0.3, saturation=0.6), # not contrast, or the background will be afected too
    GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2)),
    RandomHorizontalFlip(p=0.15),
    RandomVerticalFlip(p=0.15),
])

---

## Dataset

In [8]:
def gen(ds=[]):
    """Function to convert InfantDataset to generator"""
    for img, label, _ in ds:
        yield {
            'image': img,
            'label': label
        }

class InfantDataset(Dataset_torch):
    """Base dataset to load images and labels from the csv file + resize/pad to 224x224"""
    def __init__(self, image_dir:str, csv_path:str, nfcs_component:NFCS) -> None:
        super().__init__()
        self.data = pd.read_csv(csv_path)
        self.nfcs_component = nfcs_component
        nfcs2dir = {
            NFCS.SN: 'nasolabial_fold',
            NFCS.FS: 'forehead',
            NFCS.FP: 'eyes',
            NFCS.BA: 'mouth',
            NFCS.BE: 'mouth'
        }
        self.image_dir = Path(image_dir, nfcs2dir[nfcs_component])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        label = row[self.nfcs_component.name]
        
        # Search image file
        try:
            img_path = next(self.image_dir.glob(f"{row['image_id']}.*"))
        except StopIteration:
            raise FileNotFoundError(f"No file named {row['image_id']} found at {self.image_dir}")
        # Load image
        img = Image.open(img_path)
        # Resize keeping aspect ratio
        img = self.resize_pad(img, 224) # ViT resizes to 224x224, so we resize first to avoid distortions

        return img, label, img_path
    
    def resize_pad(self, img, smaller_side_size:int):
        # img = img.rotate(90, expand=1) # Rotation for test purposes
        # Resize
        ar = img.height/img.width # Aspect Ratio
        img = resize(img, (smaller_side_size, round(smaller_side_size/ar)) if ar > 1 else (round(smaller_side_size*ar), smaller_side_size)) # Resize to fit the smaller dimension to smaller_side_size
        # Padding
        pad_horizontal = (smaller_side_size-img.width)/2 # How much to pad horizontally
        pad_vertical = (smaller_side_size-img.height)/2 # Hor much to pad vertically
        img = pad(img, padding=(ceil(pad_horizontal), ceil(pad_vertical), int(pad_horizontal), int(pad_vertical)), padding_mode ='constant') # (left, top, right, bottom)
        return img


# Instances
torch_ds = InfantDataset(
    image_dir=masked_images_path if MASKED else non_masked_images_path,
    csv_path=csv_path,
    nfcs_component=NFCS_REGION
)

In [9]:
def transforms(batch, processor, augmentations):
    """Apply augmentations and necessary preprocessing using the processor instance"""
    inputs = processor([augmentations(x) for x in batch['image']], return_tensors='pt'),
    # For some reason, some times the output of the line above is a tuple and other times is a dictionary.. so the logic bellow is necessary
    if isinstance(inputs, tuple):
        if len(inputs) > 1:
            raise Exception(f"Unexpected len for input in transforms ({len(inputs)})")
        inputs = inputs[0]
    else:
        raise Exception("Unexpected...")
    inputs['labels'] = batch['label']
    return inputs

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

# Cast dataset from pytorch to huggingface
ds = Dataset.from_generator(gen, gen_kwargs=({'ds':torch_ds}))
# Set train test split
ds = ds.train_test_split(test_size=VALIDATION_SIZE, shuffle=False) # Shuffle false so only images in the unifesp set are used for validation
# Add augmentations and pre-process
ds['train'].set_transform(partial(transforms, processor=processor, augmentations=augmentations))
ds['test'].set_transform(partial(transforms, processor=processor, augmentations=Compose([]))) # No augmentations in the test set

c:\Users\12ped\anaconda3\envs\optuna\lib\site-packages\dill\_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'NFCS'>.
  StockPickler.save(self, obj, save_persistent_id)
c:\Users\12ped\anaconda3\envs\optuna\lib\site-packages\dill\_dill.py:414: PicklingWarning: Cannot pickle <enum 'NFCS'>: __main__.NFCS has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
# === SAVE DATASET TO DISK === #

# from tqdm import tqdm
# out_dir = Path(f'/home/phdomingues/masters/src/Labeled_split/{NFCS_REGION.name}')
# for img, label, pth in tqdm(torch_ds, total=len(ds)):
#     out_dir_label = out_dir / str(label)
#     out_dir_label.mkdir(exist_ok=True, parents=True)
#     img.save(out_dir_label / pth.name)

---

## Training

In [11]:
def compute_metrics(p, metrics):
    pred = np.argmax(p.predictions, axis=1)
    val = [metric.compute(predictions=pred, references=p.label_ids) for metric in metrics]
    return {k: v for d in val for k, v in d.items()} # Merge list of metrics into a single dictionary with all metrics

In [12]:
training_args = TrainingArguments(
    output_dir=output_dir,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    # learning_rate=config.learning_rate,
    # weight_decay=config.weight_decay,
    num_train_epochs=TRAIN_EPOCHS,
    # per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=1,
    save_total_limit=1,
    remove_unused_columns=False,
    push_to_hub=False,
    fp16=False, # float point 16 bit precision (instead of 32)
    load_best_model_at_end=True,
    metric_for_best_model=OBJECTIVE_METRIC,
    greater_is_better=GREATER_IS_BETTER,
    save_only_model=True
)

trainer = Trainer(
    model=None,
    model_init=model_init,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=partial(compute_metrics, metrics=[load_metric(m, trust_remote_code=True) for m in METRICS]),
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    tokenizer=processor,
)

c:\Users\12ped\anaconda3\envs\optuna\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\12ped\AppData\Local\Temp\ipykernel_15060\2702257867.py:26: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  compute_metrics=partial(compute_metrics, metrics=[load_metric(m, trust_remote_code=True) for m in METRICS]),
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=NUM_TRIALS,
    compute_objective=compute_objective,
)

[I 2024-07-22 18:21:27,241] A new study created in memory with name: no-name-c7f6252f-bddb-4c10-ac34-9b13dd0254a4
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/380 [00:00<?, ?it/s]

c:\Users\12ped\anaconda3\envs\optuna\lib\site-packages\transformers\models\vit\modeling_vit.py:253: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.6651, 'grad_norm': 0.06999296694993973, 'learning_rate': 0.0004656909601678784, 'epoch': 1.0}


  0%|          | 0/108 [00:00<?, ?it/s]

...

  0%|          | 0/380 [00:00<?, ?it/s]

{'loss': 0.6359, 'grad_norm': 1.182146430015564, 'learning_rate': 3.5069880339523914e-05, 'epoch': 1.0}


  0%|          | 0/108 [00:00<?, ?it/s]

[I 2024-07-22 21:38:00,468] Trial 29 pruned. 


{'eval_loss': 0.6049523949623108, 'eval_accuracy': 0.7222222222222222, 'eval_f1': 0.5588235294117647, 'eval_runtime': 3.0909, 'eval_samples_per_second': 34.941, 'eval_steps_per_second': 34.941, 'epoch': 1.0}


In [14]:
best_run

BestRun(run_id='3', objective=0.8, hyperparameters={'learning_rate': 4.003920490135503e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0}, run_summary=None)

In [15]:
study = trainer._trial.study

# Save study
optuna.copy_study(
    from_study_name=study.study_name,
    from_storage=study._storage,
    to_storage=f"sqlite:///{output_dir}/study.db",
    to_study_name="study"
)

# Save sampler state
with open(f"{output_dir}/sampler.pkl", "wb") as fout:
    pickle.dump(study.sampler, fout)
    
# Save pruner state
with open(f"{output_dir}/pruner.pkl", "wb") as fout:
    pickle.dump(study.pruner, fout)

[I 2024-07-22 21:38:01,018] A new study created in RDB with name: study


In [16]:
optuna.visualization.plot_optimization_history(study)

In [17]:
optuna.visualization.plot_param_importances(study)

In [18]:
optuna.visualization.plot_contour(study)

---

# Train the best model

In [19]:
best_run

BestRun(run_id='3', objective=0.8, hyperparameters={'learning_rate': 4.003920490135503e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0}, run_summary=None)

In [20]:
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     save_strategy='epoch',
#     evaluation_strategy='epoch',
#     logging_strategy='epoch',
#     learning_rate=best_run.hyperparameters['learning_rate'],
#     # weight_decay=config.weight_decay,
#     num_train_epochs=TRAIN_EPOCHS,
#     per_device_train_batch_size=best_run.hyperparameters['per_device_train_batch_size'],
#     per_device_eval_batch_size=1,
#     save_total_limit=1,
#     remove_unused_columns=False,
#     push_to_hub=False,
#     fp16=False, # float point 16 bit precision (instead of 32)
#     load_best_model_at_end=True,
# )

# trainer = Trainer(
#     model=None,
#     model_init=model_init,
#     args=training_args,
#     data_collator=collate_fn,
#     compute_metrics=partial(compute_metrics, metrics=[load_metric(m, trust_remote_code=True) for m in METRICS]),
#     train_dataset=ds['train'],
#     eval_dataset=ds['test'],
#     tokenizer=processor,
# )

# trainer.train()

In [21]:
# # Re-Evaluate the best model
# metrics = trainer.evaluate()
# print(metrics)

In [22]:
best_run.hyperparameters

{'learning_rate': 4.003920490135503e-05,
 'per_device_train_batch_size': 16,
 'weight_decay': 0}

In [23]:
study.trials[int(best_run.run_id)]

FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.8], datetime_start=datetime.datetime(2024, 7, 22, 18, 37, 51, 699898), datetime_complete=datetime.datetime(2024, 7, 22, 18, 44, 34, 841965), params={'learning_rate': 4.003920490135503e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0}, user_attrs={}, system_attrs={}, intermediate_values={38: 0.7878787878787878, 76: 0.7619047619047619, 114: 0.7676767676767676, 152: 0.7619047619047619, 190: 0.6666666666666666, 228: 0.8045977011494253, 266: 0.7640449438202247, 304: 0.7848101265822784, 342: 0.7906976744186046, 380: 0.7027027027027027, 418: 0.7532467532467533, 456: 0.8, 494: 0.7901234567901234, 532: 0.8, 570: 0.7901234567901234, 608: 0.7777777777777778, 646: 0.7631578947368421, 684: 0.7951807228915663, 722: 0.8048780487804879, 760: 0.8}, distributions={'learning_rate': FloatDistribution(high=0.001, log=True, low=1e-06, step=None), 'per_device_train_batch_size': CategoricalDistribution(choices=(4, 8, 16, 32)), 'weight_decay':

In [ ]:
set_split_summary = {
    'train': {
        'image_count': {
            '0': 0,
            '1': 0
        }
    },
    'test': {
        'image_count': {
            '0': 0,
            '1': 0
        }
    }
}
for ds_set in set_split_summary:
    for data in ds[ds_set]:
        set_split_summary[ds_set]['image_count'][str(data['labels'])] += 1
    total = sum(set_split_summary[ds_set]['image_count'].values())
    ratio0 = round(set_split_summary[ds_set]['image_count']['0'] / total * 100)
    set_split_summary[ds_set]['ratio'] = f'{ratio0}/{100-ratio0}'

In [24]:
summary = {
    'optuna': {
        'best_run_id': best_run.run_id,
        'objective': OBJECTIVE_METRIC,
        'direction': DIRECTION,
        'optimized_objective': best_run.objective,
        'trials': NUM_TRIALS,
        'epochs': TRAIN_EPOCHS,
        'tunned_params': best_run.hyperparameters,
    },
    'dataset': DATASET.name,
    'nfcs': NFCS_REGION.name,
    'masked': MASKED,
    'val_split': VALIDATION_SIZE,
    'train_val_details': set_split_summary,
    'model': model_name_or_path,
    'pickle_version': pickle.format_version,
    'timestamp': now.isoformat(timespec='minutes'),
    'description': DESCRIPTION
}

In [25]:
with open(f'{output_dir}/summary.json', "w") as f:
    json.dump(summary, f, indent=2)